# text-classification-model-v1
This notebook classifies website data using transfer learning starting from an existing hugging face model
* Get a model checkpoint for an encoder model 
* Use reinforcement learning to apply the model on a new classification problem (EAGER website data) with limited new trained data
* Apply new head of model to full EAGER corpus to come up with mixes of models

In [1]:
!git clone https://github.com/euphonic/EAGER.git

Cloning into 'EAGER'...
remote: Enumerating objects: 19616, done.
remote: Counting objects: 100% (308/308), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 19616 (delta 180), reused 164 (delta 101), pack-reused 19308
Receiving objects: 100% (19616/19616), 370.00 MiB | 25.37 MiB/s, done.
Resolving deltas: 100% (5898/5898), done.
Checking out files: 100% (11765/11765), done.


## Install libraries

In [ ]:
# install bson for reading mongodb data
!pip uninstall --yes bson
!pip install pymongo

WARNING: Skipping bson as it is not installed. 
Requirement already satisfied: pymongo in /databricks/python3/lib/python3.8/site-packages (4.1.1)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [2]:
# install hugging face 
!pip install transformers
!pip install datasets
!pip install DatasetDict

     |████████████████████████████████| 4.2 MB 8.4 MB/s 
     |████████████████████████████████| 6.6 MB 37.4 MB/s 
     |████████████████████████████████| 84 kB 2.6 MB/s 
     |████████████████████████████████| 596 kB 37.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 342 kB 7.3 MB/s 
     |████████████████████████████████| 136 kB 51.6 MB/s 
     |████████████████████████████████| 212 kB 50.8 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 127 kB 51.0 MB/s 
     |████████████████████████████████| 271 kB 53.1 MB/s 
     |████████████████████████████████| 144 kB 52.1 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ER

In [27]:
import tensorflow as tf
import numpy as np
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification

import gzip
import tarfile
import shutil
import bson
import pandas as pd

## Canned Examples

In [ ]:
print(pipeline('sentiment-analysis')('This application looks promising'))

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
[{'label': 'POSITIVE', 'score': 0.9940495491027832}]

In [28]:
# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = dict(tokenizer(sequences, padding=True, truncation=True, return_tensors="tf"))

# This is new
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
labels = tf.convert_to_tensor([1, 1])
model.train_on_batch(batch, labels)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2.3841855067985307e-07

## EAGER test

In [ ]:
# EAGER data -- unpack
tar_dir = '/dbfs/FileStore/eager/'
tar_file = tar_dir + "FirmDB_about2_20190131.tar"
print (tar_file)

/dbfs/FileStore/eager/FirmDB_about2_20190131.tar

In [ ]:
# tar = tarfile.open(tar_file)
# tar.extractall(tar_dir)
# tar.close()

In [ ]:
# gunzip about us pages
ungzip_file = tar_dir + "FirmDB_about2_20190131/pages_ABOUT2.bson"
gzip_file = ungzip_file + ".gz"
print (gzip_file)

/dbfs/FileStore/eager/FirmDB_about2_20190131/pages_ABOUT2.bson.gz

In [ ]:
with gzip.open(gzip_file, 'rb') as f_in:
    with open(ungzip_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
with open(ungzip_file,'rb') as f:
    about_pages = bson.decode_all(f.read())

In [ ]:
about_pages[0]['full_text']

Out[11]: ['About Xintec',
 'Introduction',
 'Vision & Core Value',
 'Management Team',
 'Milestone',
 'Press Center',
 'Quality & ESH Policy',
 'Business Ethics',
 'Contact Us',
 'Products / Services',
 'Optical Sensor CSP',
 'PPI & 3D Chip Scale Package',
 'MEMS & Sensor Packaging',
 'Design and Modeling Services',
 'Investor Relation',
 'Company Profile',
 'Monthly Revenue',
 'Quarterly Results',
 'Financial Reports',
 "Shareholders' Service",
 'Material Information',
 'TSE MOPS',
 'Calendar of Events',
 'FAQ',
 'Human Resources',
 'Training & Development',
 'Work & Life',
 'Recruting',
 'CSR',
 'Message from Chairman',
 'Corporate Governance',
 'Stakeholder Engagement',
 'Work Environment & Social Contribution',
 'CSR Policy and Reports',
 '繁中',
 'About Xintec',
 '\r\n >\r\n ',
 'About Xintec',
 '\r\n > Vision & Core Value\r\n ',
 'Introduction',
 'Vision & Core Value',
 'Management Team',
 'Milestone',
 'Press Center',
 'Quality & ESH Policy',
 'Business Ethics',
 'Contact Us',
 'Vision & Core Value',
 'Company Vision',
 'Be the best advanced wafer level packaging foundry service provider.',
 'Company Core Value',
 'Integrity: Being honest to customers, shareholder and ourselves. Willing to commit and always fulfill commitment.',
 'Innovation: Passionately generate and realize value adding ideas.',
 'Customer Orientation: Offer customer desired technology, robust production, excellent quality and premium service.',
 'Copyright © 2014 Xintec Inc. ALL rights reserved.\r\n \xa0|\xa0\r\n ',
 'Site Map',
 '\r\n \xa0|\xa0\r\n ',
 'Contact Us',
 '\r\n This website is best viewed by Internet Explorer 8+']

### Zero-shot classification

In [ ]:
firm_file_location = '/dbfs/FileStore/eager/about/Zygo.txt'
txt_file = open(firm_file_location, "r")
content_list = txt_file.readlines()
print(content_list)

['Metrology\n', '3Optical Profilers\n', 'Laser Interferometers\n', 'Position Metrology Systems\n', 'Custom Metrology Solutions\n', 'Trade-in & Upgrade Program\n', 'Optics\n', 'Optical Components Group\n', 'Electro-Optics Group\n', '- Engineering & Product Development\n', '- Electro-Optical Assembly\n', '- Opto-Mechanical Assembly\n', '- Turnkey System Manufacturing\n', 'Markets & Applications\n', '[close]\n', '\n', 'By continuing to use this site, you agree to our \n', 'Privacy and Cookie Policy\n', '. \n', 'OK\n', "What's New at ZYGO?\n", '››\n', 'News:\n', ' Launches Latest Generation of ZeGage Non-Contact 3Optical Profilers\n', '››\n', 'Blog:\n', ' Using Grazing Incidence Interferometry to Measure Large or Non-Specular Surfaces\n', '››\n', 'Blog:\n', ' Process Metrology Helps Ensure Quality of Smartphone Camera Lenses\n', '››\n', 'Blog:\n', ' Precision Lead Angle Metrology Improves Shaft Seals and Engine Efficiency\n', '››\n', 'Blog:\n', ' Precision Gear Tooth Metrology Helps Automaker Silence Humming Differential\n', '››\n', 'Blog:\n', ' Measuring Roughness on Super-Smooth Surfaces\n', '››\n', 'Blog:\n', ' New Smart Setup Feature Streamlines Precision Surface Metrology\n', '››\n', 'Blog:\n', ' Think you know ZYGO? Check out these six surprising facts and statistics...\n', 'Visit at these\n', 'Upcoming Events:\n', 'MD&West\n', '02/05\n', 'Photonics West\n', '02/05\n', 'Advanced Lithography\n', '02/26\n', 'Houstex\n', '02/26\n', 'Defense & Sensing\n', '04/16\n', 'Ann. Mtg. & Exhibt\n', '05/19\n', 'West\n', '07/09\n', 'Optics & Photonics\n', '08/13\n', 'Optifab\n', '10/15\n', '(Click for a complete list)\n', 'Privacy and Cookie Policy\n', 'Privacy Policy Inquiry Form\n', 'Unsubscribe\n', '×\n', 'Privacy Policy InquiryAbout Us\n', 'Company Profile\n', 'News Releases\n', 'Trade Show Schedule\n', 'Manufacturing Locations\n', 'Sales/Service Offices\n', 'How to Get to Patent Information\n', 'Registrations\n', 'Conflict Minerals Policy\n', 'Privacy and Cookie Policy\n', 'Terms & Conditions\n', "ZYGO's booth at Photonics West\n", 'Micro-Seminar in Progress\n', 'free\n', '~ 2019 ~\n', 'MD&West\n', 'Photonics West 2019 Exhibition\n', 'W3 Fair+Convention 2019\n', 'Advanced Lithography\n', 'Houstex\n', 'Defense & Sensing\n', 'CONTROL\n', 'Annual Meeting & Exhibition\n', 'EUSPEN\n', 'LASER\n', 'West\n', 'Optics & Photonics\n', 'Optifab 2019\n', 'IMTS\n', '[close]\n', '\n', 'By continuing to use this site, you agree to our \n', 'Privacy and Cookie Policy\n', '. \n', 'OK\n', "What's New at ZYGO?\n", '››\n', 'News:\n', ' Launches Latest Generation of ZeGage Non-Contact 3Optical Profilers\n', '››\n', 'Blog:\n', ' Using Grazing Incidence Interferometry to Measure Large or Non-Specular Surfaces\n', '››\n', 'Blog:\n', ' Process Metrology Helps Ensure Quality of Smartphone Camera Lenses\n', '››\n', 'Blog:\n', ' Precision Lead Angle Metrology Improves Shaft Seals and Engine Efficiency\n', '››\n', 'Blog:\n', ' Precision Gear Tooth Metrology Helps Automaker Silence Humming Differential\n', '››\n', 'Blog:\n', ' Measuring Roughness on Super-Smooth Surfaces\n', '››\n', 'Blog:\n', ' New Smart Setup Feature Streamlines Precision Surface Metrology\n', '››\n', 'Blog:\n', ' Think you know ZYGO? Check out these six surprising facts and statistics...\n', 'Visit at these\n', 'Upcoming Events:\n', 'MD&West\n', '02/05\n', 'Photonics West\n', '02/05\n', 'Advanced Lithography\n', '02/26\n', 'Houstex\n', '02/26\n', 'Defense & Sensing\n', '04/16\n', 'Ann. Mtg. & Exhibt\n', '05/19\n', 'West\n', '07/09\n', 'Optics & Photonics\n', '08/13\n', 'Optifab\n', '10/15\n', '(Click for a complete list)\n', 'Privacy and Cookie Policy\n', 'Privacy Policy Inquiry Form\n', 'Unsubscribe\n', '×\n', 'Privacy Policy InquiryOptical Surface Profilers\n', 'Products\n', '3Optical Profilers\n', 'Laser Interferometers\n', 'Position Metrology Systems\n', 'Custom Metrology Solutions\n', 'Trade-in & Upgrade Program\n', 'Optical Surface Profilers\n', 'ZeGage™ Pro\xa0 \n', 'New!\n', 'NewView™ 90

In [ ]:
content_list[6]

Out[13]: 'Optics\n'

In [ ]:
classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)
Downloading: 0%| | 0.00/1.13k [00:00<?, ?B/s]
Downloading: 0%| | 0.00/1.52G [00:00<?, ?B/s]
Downloading: 0%| | 0.00/26.0 [00:00<?, ?B/s]
Downloading: 0%| | 0.00/878k [00:00<?, ?B/s]
Downloading: 0%| | 0.00/446k [00:00<?, ?B/s]
Downloading: 0%| | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
candidate_labels = ["business", "marketing", "manufacturing", "research", "engineering"]

for i in range(6,10):
  res = classifier(content_list[i], candidate_labels)
  print (res)

{'sequence': 'Optics\n', 'labels': ['business', 'marketing', 'research', 'engineering', 'manufacturing'], 'scores': [0.2477385550737381, 0.24553173780441284, 0.21116505563259125, 0.170455202460289, 0.12510938942432404]}
{'sequence': 'Optical Components Group\n', 'labels': ['business', 'engineering', 'research', 'manufacturing', 'marketing'], 'scores': [0.7122617959976196, 0.10993733257055283, 0.09078780561685562, 0.05439535900950432, 0.03261769190430641]}
{'sequence': 'Electro-Optics Group\n', 'labels': ['business', 'research', 'engineering', 'marketing', 'manufacturing'], 'scores': [0.6276072859764099, 0.19434507191181183, 0.10650159418582916, 0.03894878178834915, 0.03259727358818054]}
{'sequence': '- Engineering & Product Development\n', 'labels': ['engineering', 'business', 'research', 'manufacturing', 'marketing'], 'scores': [0.663918673992157, 0.17188161611557007, 0.11716794222593307, 0.036675307899713516, 0.010356508195400238]}

In [ ]:
classifier("I havern't a dog.")

Out[31]: [{'label': 'LABEL_0', 'score': 0.9532511234283447}]

In [ ]:
classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")

for i in range(0, len(content_list)):
  print ((content_list[i]))
  # res = classifier(content_list[i])
  # print (res)

Downloading: 0%| | 0.00/490 [00:00<?, ?B/s]
Downloading: 0%| | 0.00/255M [00:00<?, ?B/s]
Downloading: 0%| | 0.00/48.0 [00:00<?, ?B/s]
Downloading: 0%| | 0.00/226k [00:00<?, ?B/s]
Downloading: 0%| | 0.00/112 [00:00<?, ?B/s]
Metrology

3Optical Profilers

Laser Interferometers

Position Metrology Systems

Custom Metrology Solutions

Trade-in & Upgrade Program

Optics

Optical Components Group

Electro-Optics Group

- Engineering & Product Development

- Electro-Optical Assembly

- Opto-Mechanical Assembly

- Turnkey System Manufacturing

Markets & Applications

[close]



By continuing to use this site, you agree to our 

Privacy and Cookie Policy

. 

OK

What's New at ZYGO?

››

News:

 Launches Latest Generation of ZeGage Non-Contact 3Optical Profilers

››

Blog:

 Using Grazing Incidence Interferometry to Measure Large or Non-Specular Surfaces

››

Blog:

 Process Metrology Helps Ensure Quality of Smartphone Camera Lenses

››

Blog:

 Precision Lead Angle Metrology Improves Shaft Seals and Engine Efficiency

››

Blog:

 Precision Gear Tooth Metrology Helps Automaker Silence Humming Differential

››

Blog:

 Measuring Roughness on Super-Smooth Surfaces

››

Blog:

 New Smart Setup Feature Streamlines Precision Surface Metrology

››

Blog:

 Think you know ZYGO? Check out these six surprising facts and statistics...

Visit at these

Upcoming Events:

MD&West

02/05

Photonics West

02/05

Advanced Lithography

02/26

Houstex

02/26

Defense & Sensing

04/16

Ann. Mtg. & Exhibt

05/19

West

07/09

Optics & Photonics

08/13

Optifab

10/15

(Click for a complete list)

Privacy and Cookie Policy

Privacy Policy Inquiry Form

Unsubscribe

×

Privacy Policy InquiryAbout Us

Company Profile

News Releases

Trade Show Schedule

Manufacturing Locations

Sales/Service Offices

How to Get to Patent Information

Registrations

Conflict Minerals Policy

Privacy and Cookie Policy

Terms & Conditions

ZYGO's booth at Photonics West

Micro-Seminar in Progress

free

~ 2019 ~

MD&West

Photonics West 2019 Exhibition

W3 Fair+Convention 2019

Advanced Lithography

Houstex

Defense & Sensing

CONTROL

Annual Meeting & Exhibition

EUSPEN

LASER

West

Optics & Photonics

Optifab 2019

IMTS

[close]



By continuing to use this site, you agree to our 

Privacy and Cookie Policy

. 

OK

What's New at ZYGO?

››

News:

 Launches Latest Generation of ZeGage Non-Contact 3Optical Profilers

››

Blog:

 Using Grazing Incidence Interferometry to Measure Large or Non-Specular Surfaces

››

Blog:

 Process Metrology Helps Ensure Quality of Smartphone Camera Lenses

››

Blog:

 Precision Lead Angle Metrology Improves Shaft Seals and Engine Efficiency

››

Blog:

 Precision Gear Tooth Metrology Helps Automaker Silence Humming Differential

››

Blog:

 Measuring Roughness on Super-Smooth Surfaces

››

Blog:

 New Smart Setup Feature Streamlines Precision Surface Metrology

››

Blog:

 Think you know ZYGO? Check out these six surprising facts and statistics...

Visit at these

Upcoming Events:

MD&West

02/05

Photonics West

02/05

Advanced Lithography

02/26

Houstex

02/26

Defense & Sensing

04/16

Ann. Mtg. & Exhibt

05/19

West

07/09

Optics & Photonics

08/13

Optifab

10/15

(Click for a complete list)

Privacy and Cookie Policy

Privacy Policy Inquiry Form

Unsubscribe

×

Privacy Policy InquiryOptical Surface Profilers

Products

3Optical Profilers

Laser Interferometers

Position Metrology Systems

Custom Metrology Solutions

Trade-in & Upgrade Program

Optical Surface Profilers

ZeGage™ Pro  

New!

NewView™ 9000

Nexview™NX2

Nomad™

Compass™

APM650™

Mx™ Software

Profiler Objectives

Optical Profiler Basics

Surface Roughness Measurement

Surface Profiling

3Optical Microscopy

3Interference Microscopy

ZYGO's 3Optical Profiler instruments enable precise, quantitative, ISO-compliant, 

non-contact

 surface measurement and characterization of micro- and nano-scale surface features, capturing 

up to two million data points

in just seconds


### Garbage classifier
keep test == 1, discard == 0

In [14]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

In [11]:
firm_file_location = '/content/EAGER/data/modeling/garbage/garbage_classifier_input.csv'
input_df = pd.read_csv(firm_file_location)

In [12]:
non_null_df = input_df[~ input_df['sample_text'].isnull() ]
non_null_df

,sample_text,keep_text
0,Our Management,0
1,Latest Press Releases,0
2,On-Going Clinical Studies on Very Low Nicotine...,0
3,Links to the ‚ÄúMiracle Plant‚Äù,0
4,This advisory note presents the conclusions an...,1
...,...,...
4057,How It Works,0
4058,Read Article,0
4059,Tank Farms,0
4060,Industries Served,0


In [16]:
dataset = Dataset.from_pandas(non_null_df, split='train')
dataset.cast_column("keep_text", datasets.Value('int8'))

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['sample_text', 'keep_text', '__index_level_0__'],
    num_rows: 4055
})

In [19]:
# 80% train, 20% test + validation
train_test_dataset = dataset.train_test_split(test_size=0.2)
# Split the 20% test + valid in half test, half valid
test_valid_dataset = train_test_dataset['test'].train_test_split(test_size=0.2)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = datasets.DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_valid_dataset['test'],
    'valid': test_valid_dataset['train']})

In [20]:
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['sample_text', 'keep_text', '__index_level_0__'],
        num_rows: 3244
    })
    test: Dataset({
        features: ['sample_text', 'keep_text', '__index_level_0__'],
        num_rows: 163
    })
    valid: Dataset({
        features: ['sample_text', 'keep_text', '__index_level_0__'],
        num_rows: 648
    })
})

In [25]:
def tokenize_function(x):
  return tokenizer(x["sample_text"], truncation=True, max_length=100)

In [29]:
tokenized_dataset = train_test_valid_dataset.map(tokenize_function, batched=True, batch_size=2000)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [32]:
samples = tokenized_dataset["train"].to_dict()
samples = {k: v for k, v in samples.items() if k not in ["idx", "sample_text"]}
set([len(x) for x in samples["input_ids"]])

{2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100}

In [33]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length", max_length=100, return_tensors="tf")

In [34]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'__index_level_0__': TensorShape([3244]),
 'attention_mask': TensorShape([3244, 100]),
 'input_ids': TensorShape([3244, 100]),
 'keep_text': TensorShape([3244]),
 'token_type_ids': TensorShape([3244, 100])}

In [35]:
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols="keep_text",
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_dataset["valid"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols="keep_text",
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [36]:
tf_train_dataset

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(8, None), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(8, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(8, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(8,), dtype=tf.int64, name=None))>

In [37]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

In [39]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 5
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [40]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=num_epochs)

Epoch 1/5
405/405 [==============================] - 3497s 9s/step - loss: 0.2199 - accuracy: 0.9108 - val_loss: 0.2131 - val_accuracy: 0.9182
Epoch 2/5
405/405 [==============================] - 3465s 9s/step - loss: 0.1423 - accuracy: 0.9451 - val_loss: 0.1767 - val_accuracy: 0.9290
Epoch 3/5
405/405 [==============================] - 3466s 9s/step - loss: 0.0717 - accuracy: 0.9787 - val_loss: 0.2242 - val_accuracy: 0.9228
Epoch 4/5
405/405 [==============================] - 3489s 9s/step - loss: 0.0358 - accuracy: 0.9895 - val_loss: 0.3370 - val_accuracy: 0.9136
Epoch 5/5
405/405 [==============================] - 3661s 9s/step - loss: 0.0252 - accuracy: 0.9938 - val_loss: 0.3281 - val_accuracy: 0.9213


In [43]:
model.save('/content/EAGER/models/garbage_classifier_v1')

INFO:tensorflow:Assets written to: /content/EAGER/models/garbage_classifier_v1/assets


INFO:tensorflow:Assets written to: /content/EAGER/models/garbage_classifier_v1/assets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')